In [157]:
import pandas as pd
import shapefile
import numpy as np

In [158]:
county_list = pd.read_excel("..\Public Data\VARIABLE for coder X.xlsx")
shp = shapefile.Reader("..\..\Local\Export_Output.shp")
holc = shapefile.Reader(r"..\..\Local\Tracts_2020_HOLC\Tracts_2020_HOLC.shp")
intersection = shapefile.Reader(r"..\..\Local\neigb_level_park_coverage.shp")
len(shp)

42074

In [159]:
len(intersection)

39603

In [160]:
df=pd.DataFrame([holc.record(i).as_dict() for i in range(len(holc))])
intersection_df = pd.DataFrame([intersection.record(i).as_dict() for i in range(len(intersection))])

In [161]:
df["COUNTYGISJOIN"] = df["GISJOIN"].apply(lambda x: x[0:8])
intersection_df["COUNTYGISJOIN"] = intersection_df["GISJOIN"].apply(lambda x: x[0:8])

In [162]:
df=df.reset_index(drop=False)
df.head(3)

,index,GISJOIN,neighborho,SUM_Perc,FIRST_holc,MAX_state,MAX_city,Shape_Leng,Shape_Area,COUNTYGISJOIN
0,0,G0100730000100,180,0.092969,D,AL,Birmingham,3336.386272,7.095820e+05,G0100730
1,1,G0100730000100,181,0.161693,D,AL,Birmingham,7731.380924,1.234060e+06,G0100730
2,2,G0100730000100,182,0.170326,D,AL,Birmingham,6111.241095,1.300062e+06,G0100730


In [163]:
intersection_df=intersection_df.rename(columns={"FID_Tracts":"Holc_key",
                                "Shape_Area":"Park_Area"})
intersection_df.head(3)

,OBJECTID,Holc_key,GISJOIN,FIRST_holc,MAX_state,MAX_city,Shape_Leng,Park_Area,COUNTYGISJOIN
0,1,0,G0100730000100,D,AL,Birmingham,2213.280164,179020.783290,G0100730
1,2,1,G0100730000100,D,AL,Birmingham,1628.851990,42287.696567,G0100730
2,3,2,G0100730000100,D,AL,Birmingham,5648.456758,459488.376606,G0100730


In [164]:
region_park_area = df.merge(intersection_df,left_on="index",right_on="Holc_key",how="left")
region_park_area = region_park_area[["index","COUNTYGISJOIN_x","FIRST_holc_x","MAX_state_x","Shape_Area","Park_Area"]]
region_park_area["Park_Area"] = region_park_area["Park_Area"].fillna(0)
region_park_area["Park_Area_Ratio"]=region_park_area["Park_Area"]/region_park_area["Shape_Area"]
region_park_area

,index,COUNTYGISJOIN_x,FIRST_holc_x,MAX_state_x,Shape_Area,Park_Area,Park_Area_Ratio
0,0,G0100730,D,AL,7.095820e+05,1.790208e+05,0.252290
1,1,G0100730,D,AL,1.234060e+06,4.228770e+04,0.034267
2,2,G0100730,D,AL,1.300062e+06,4.594884e+05,0.353436
3,3,G0100730,C,AL,1.971179e+06,1.831822e+06,0.929303
4,4,G0100730,B,AL,3.204816e+05,2.523207e+05,0.787317
...,...,...,...,...,...,...,...
42069,42069,G5501390,D,WI,1.607568e+05,1.607568e+05,1.000000
42070,42070,G5501390,C,WI,5.770248e+05,5.242086e+05,0.908468
42071,42071,G5501390,C,WI,1.097678e+05,3.052323e+04,0.278071
42072,42072,G5501390,D,WI,4.189811e+05,2.822071e+05,0.673556


In [165]:
np.sum(region_park_area.isna())

index              0
COUNTYGISJOIN_x    0
FIRST_holc_x       0
MAX_state_x        0
Shape_Area         0
Park_Area          0
Park_Area_Ratio    0
dtype: int64

In [166]:
region_park_area[region_park_area["Park_Area"]==0]

,index,COUNTYGISJOIN_x,FIRST_holc_x,MAX_state_x,Shape_Area,Park_Area,Park_Area_Ratio
8,8,G0100730,D,AL,343664.496413,0.0,0.0
9,9,G0100730,D,AL,318867.956987,0.0,0.0
21,21,G0100730,D,AL,3501.858498,0.0,0.0
37,37,G0100730,D,AL,15744.942229,0.0,0.0
59,59,G0100730,C,AL,61103.166980,0.0,0.0
...,...,...,...,...,...,...,...
41986,41986,G5501010,C,WI,96838.993301,0.0,0.0
42007,42007,G5501010,B,WI,14009.838906,0.0,0.0
42010,42010,G5501010,B,WI,522.930430,0.0,0.0
42033,42033,G5501390,D,WI,579234.374135,0.0,0.0


## Filter Out Places That Is Classified as A or B Areas, and Find Which Neighborhoods Have Less than 100% 10-min Walk Park Access.

In [167]:
region_park_area = region_park_area[region_park_area["FIRST_holc_x"].isin(["C","D","E"])]
county_min_park_ratios = region_park_area.groupby("COUNTYGISJOIN_x").agg({"Park_Area_Ratio":"min"}).reset_index(drop=False)
county_min_park_ratios = county_min_park_ratios[county_min_park_ratios["Park_Area_Ratio"]<1]
county_min_park_ratios["Insufficient_Park_Access"] = (county_min_park_ratios["Park_Area_Ratio"]<1).replace({True:1,False:0})
county_min_park_ratios

,COUNTYGISJOIN_x,Park_Area_Ratio,Insufficient_Park_Access
0,G0100730,0.000000,1
1,G0100970,0.000000,1
2,G0101010,0.000000,1
3,G0400130,0.000000,1
4,G0501190,0.000000,1
...,...,...,...
239,G5500590,0.000000,1
240,G5500790,0.000000,1
241,G5501010,0.000000,1
242,G5501330,0.922921,1


In [168]:
county_list

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME
0,Alabama,G0100010,41.0,1,10,Autauga,NaN
1,Alabama,G0100030,41.0,1,30,Baldwin,NaN
2,Alabama,G0100050,41.0,1,50,Barbour,NaN
3,Alabama,G0100070,41.0,1,70,Bibb,NaN
4,Alabama,G0100090,41.0,1,90,Blount,NaN
...,...,...,...,...,...,...,...
3253,Wyoming,G5500650,68.0,56,390,Teton,NaN
3254,Wyoming,G5500670,68.0,56,410,Uinta,NaN
3255,Wyoming,G5500690,68.0,56,430,Washakie,NaN
3256,Wyoming,G5500710,68.0,56,450,Weston,NaN


### Mistake in Dr.AJ's Table! Correction is Needed.

In [169]:
def GISJOIN_correction(sx,x):
    sx = str(sx)
    x = str(x)
    result = "G"
    if len(sx)==1:
        sx = "0"+sx
    if len(x)==1:
        x = "0000"+x
    elif len(x)==2:
        x = "000"+x
    elif len(x)==3:
        x = "00"+x
    elif len(x)==4:
        x = "0"+x
    assert(len(result+sx+x)==8)
    return result + sx+x

In [170]:
county_list["corrected_GISJOIN"] = county_list.apply(lambda x: GISJOIN_correction(x["STATEFIPS"],x["COUNTYFIPS"]),axis=1)
county_list

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,corrected_GISJOIN
0,Alabama,G0100010,41.0,1,10,Autauga,NaN,G0100010
1,Alabama,G0100030,41.0,1,30,Baldwin,NaN,G0100030
2,Alabama,G0100050,41.0,1,50,Barbour,NaN,G0100050
3,Alabama,G0100070,41.0,1,70,Bibb,NaN,G0100070
4,Alabama,G0100090,41.0,1,90,Blount,NaN,G0100090
...,...,...,...,...,...,...,...,...
3253,Wyoming,G5500650,68.0,56,390,Teton,NaN,G5600390
3254,Wyoming,G5500670,68.0,56,410,Uinta,NaN,G5600410
3255,Wyoming,G5500690,68.0,56,430,Washakie,NaN,G5600430
3256,Wyoming,G5500710,68.0,56,450,Weston,NaN,G5600450


## Join the Park Access Table to Dr.AJ's Table & Create Label

In [171]:
labeled=county_list.merge(county_min_park_ratios,left_on="corrected_GISJOIN",right_on="COUNTYGISJOIN_x",how="left")

In [172]:
labeled["Insufficient_Park_Access"]=labeled["Insufficient_Park_Access"].fillna(0)
labeled=labeled.drop(columns=["COUNTYGISJOIN_x","Park_Area_Ratio","VARIABLE NAME"])
labeled=labeled.rename(columns={"Insufficient_Park_Access":"EPLRA1"})
labeled["EPLRA1"]=labeled["EPLRA1"].astype(int)
labeled.sum()

STATE                AlabamaAlabamaAlabamaAlabamaAlabamaAlabamaAlab...
GISJOIN              G0100010G0100030G0100050G0100070G0100090G01001...
STATEICP                                                      132120.0
STATEFIPS                                                       101400
COUNTYFIPS                                                     3242168
ALLCOUNTIES          AutaugaBaldwinBarbourBibbBlountBullockButlerCa...
corrected_GISJOIN    G0100010G0100030G0100050G0100070G0100090G01001...
EPLRA1                                                             240
dtype: object

In [173]:
labeled[labeled["EPLRA1"]==1]

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,corrected_GISJOIN,EPLRA1
36,Alabama,G0100730,41.0,1,730,Jefferson,G0100730,1
48,Alabama,G0100970,41.0,1,970,Mobile,G0100970,1
50,Alabama,G0101010,41.0,1,1010,Montgomery,G0101010,1
126,Arizona,G0250015,61.0,4,130,Maricopa,G0400130,1
193,Arkansas,G0500010,42.0,5,1190,Pulaski,G0501190,1
...,...,...,...,...,...,...,...,...
3191,Wisconsin,G5400510,25.0,55,590,Kenosha,G5500590,1
3202,Wisconsin,G5400730,25.0,55,790,Milwaukee,G5500790,1
3213,Wisconsin,G5400950,25.0,55,1010,Racine,G5501010,1
3229,Wisconsin,G5500170,25.0,55,1330,Waukesha,G5501330,1


In [174]:
labeled.to_csv(".\Results\EPLRA1.csv")

In [175]:
labeled

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,corrected_GISJOIN,EPLRA1
0,Alabama,G0100010,41.0,1,10,Autauga,G0100010,0
1,Alabama,G0100030,41.0,1,30,Baldwin,G0100030,0
2,Alabama,G0100050,41.0,1,50,Barbour,G0100050,0
3,Alabama,G0100070,41.0,1,70,Bibb,G0100070,0
4,Alabama,G0100090,41.0,1,90,Blount,G0100090,0
...,...,...,...,...,...,...,...,...
3253,Wyoming,G5500650,68.0,56,390,Teton,G5600390,0
3254,Wyoming,G5500670,68.0,56,410,Uinta,G5600410,0
3255,Wyoming,G5500690,68.0,56,430,Washakie,G5600430,0
3256,Wyoming,G5500710,68.0,56,450,Weston,G5600450,0
